# カタカナ15文字を自動識別するモデルの構築
# 課題投稿用Notebook
* これは課題投稿用ファイルです。
* このsubmit_katakana.ipynbファイルを編集し、学習済みモデルなどの必要ファイルを同じフォルダにおき、そのフォルダをzipしたものを投稿してください。

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pickle
import glob
import os,sys
import util
from common.optimizer import Adam
from collections import OrderedDict
from common.layers16 import *

In [14]:
def makedataset():
    """
    データセットをつくる関数です。
    自由に編集してください。
    """
    
    # 次の行は変更しないこと
    test_data= util.loaddata()
    
    # 以下は自由に編集しても構いません
    # 必要な前処理をここに記述してください  
    
    # 正規化
    #test_data = (test_data - test_data.min()) / test_data.max()
    #test_data = test_data.astype('float32')
    test_data = test_data / test_data.max()
    test_data = test_data.astype('float32')

    # 配列形式変更
    test_data = test_data.reshape(-1, 1, 28, 28)

    return test_data


def func_predict(test_data, test_label):
    """
    予測する関数
    data : 画像データ
    return loss, accuracy
    引数とreturn以外は、自由に編集してください    
    """
    
    # 以下を自由に編集してください
    with open("katakana_model_params.pickle", "rb") as f:
        model_params = pickle.load(f)
        
    with open("katakana_model_moving_save.pickle", "rb") as f:
        model_moving = pickle.load(f)
    
    class BatchNormalization:
        def __init__(self, gamma, beta, rho=0.9, moving_mean=None, moving_var=None):
            self.gamma = gamma # スケールさせるためのパラメータ, 学習によって更新させる.
            self.beta = beta # シフトさせるためのパラメータ, 学習によって更新させる
            self.rho = rho # 移動平均を算出する際に使用する減衰率

            # 予測時に使用する平均と分散
            self.moving_mean = moving_mean # muの移動平均
            self.moving_var = moving_var        # varの移動平均

            # 計算中に算出される値を保持しておく変数群
            self.batch_size = None
            self.x_mu = None
            self.x_std = None        
            self.std = None
            self.dgamma = None
            self.dbeta = None

        def forward(self, x, train_flg=True):
            """
            順伝播計算
            x :  Conv層の場合は4次元、全結合層の場合は2次元  
            """
            if x.ndim == 4:
                """
                画像形式の場合
                """
                N, C, H, W = x.shape
                x = x.transpose(0, 2, 3, 1) # NHWCに入れ替え
                x = x.reshape(N*H*W, C) # (N*H*W,C)の2次元配列に変換
                out = self.__forward(x, train_flg)
                out = out.reshape(N, H, W, C)# 4次元配列に変換
                out = out.transpose(0, 3, 1, 2) # 軸をNCHWに入れ替え
            elif x.ndim == 2:
                """
                画像形式以外の場合
                """
                out = self.__forward(x, train_flg)           

            return out

        def __forward(self, x, train_flg, epsilon=1e-8):
            """
            x : 入力. n×dの行列. nはあるミニバッチのバッチサイズ. dは手前の層のノード数
            """
            #if (self.moving_mean is None) or (self.moving_var is None):
            #    N, D = x.shape
            #    self.moving_mean = np.zeros(D)
            #    self.moving_var = np.zeros(D)

            x_mu = x - self.moving_mean # n*d行列
            x_std = x_mu / np.sqrt(self.moving_var + epsilon) # n*d行列

            # gammaでスケールし、betaでシフトさせる
            out = self.gamma * x_std + self.beta # n*d行列
            return out

        def backward(self, dout):
            """
            逆伝播計算
            dout : Conv層の場合は4次元、全結合層の場合は2次元  
            """
            if dout.ndim == 4:
                """
                画像形式の場合
                """            
                N, C, H, W = dout.shape
                dout = dout.transpose(0, 2, 3, 1) # NHWCに入れ替え
                dout = dout.reshape(N*H*W, C) # (N*H*W,C)の2次元配列に変換
                dx = self.__backward(dout)
                dx = dx.reshape(N, H, W, C)# 4次元配列に変換
                dx = dx.transpose(0, 3, 1, 2) # 軸をNCHWに入れ替え
            elif dout.ndim == 2:
                """
                画像形式以外の場合
                """
                dx = self.__backward(dout)

            return dx

        def __backward(self, dout):

            """
            ここを完成させるには、計算グラフを理解する必要があり、実装にかなり時間がかかる.
            よって、この関数を完成させる作業は、宿題とする.
            """

            # betaの勾配
            dbeta = dout.sum(axis=0)

            # gammaの勾配(n方向に合計)
            dgamma = np.sum(self.x_std * dout, axis=0)

            # Xstdの勾配
            a1 = self.gamma * dout

            # Xmuの勾配
            a2 = a1 / self.std

            # 標準偏差の逆数の勾配(n方向に合計)
            a3 = a1 * self.x_mu 

            # 標準偏差の勾配
            a4 = -(np.sum(a3, axis=0) ) / (self.std * self.std)

            # 分散の勾配
            a5 = 0.5 * a4 / self.std

            # Xmuの2乗の勾配
            a6 = a5 / self.batch_size

            # Xmuの勾配
            a7 = 2.0  * self.x_mu * a6

            # muの勾配
            a8 = -(a2+a7)

            # Xの勾配
            dx = a2 + a7 +  np.sum(a8, axis=0) / self.batch_size # 第3項はn方向に平均

            self.dgamma = dgamma
            self.dbeta = dbeta

            return dx
    
    class SimpleConvNet:
        def __init__(self, input_dim1=(1, 28, 28),input_dim2=(32, 12, 12),
                     conv_param={'filter_num':32, 'filter_size':5, 'pad':0, 'stride':1},
                     pool_param={'pool_size':2, 'pad':0, 'stride':2},
                     hidden_size=500, output_size=15, weight_init_std=0.01):
            """
            input_size : tuple, 入力の配列形状(チャンネル数、画像の高さ、画像の幅)
            conv_param : dict, 畳み込みの条件
            pool_param : dict, プーリングの条件
            hidden_size : int, 隠れ層のノード数
            output_size : int, 出力層のノード数
            weight_init_std ： float, 重みWを初期化する際に用いる標準偏差
            """

            filter_num = conv_param['filter_num']
            filter_size = conv_param['filter_size']
            filter_pad = conv_param['pad']
            filter_stride = conv_param['stride']

            pool_size = pool_param['pool_size']
            pool_pad = pool_param['pad']
            pool_stride = pool_param['stride']

            input_size1 = input_dim1[1]
            input_size2 = input_dim2[1]
            conv_output_size1 = (input_size1 + 2*filter_pad - filter_size) // filter_stride + 1 # 畳み込み後のサイズ(H,W共通)
            pool_output_size1 = (conv_output_size1 + 2*pool_pad - pool_size) // pool_stride + 1 # プーリング後のサイズ(H,W共通)
            pool_output_pixel1 = filter_num * pool_output_size1 * pool_output_size1 # プーリング後のピクセル総数

            conv_output_size2 = (input_size2 + 2*filter_pad - filter_size) // filter_stride + 1 # 畳み込み後のサイズ(H,W共通)
            pool_output_size2 = (conv_output_size2 + 2*pool_pad - pool_size) // pool_stride + 1 # プーリング後のサイズ(H,W共通)
            pool_output_pixel2 = filter_num * pool_output_size2 * pool_output_size2 # プーリング後のピクセル総数


            # 重みの初期化
            self.params = {}
            std = weight_init_std
            self.params['W1']=model_params['W1']
            self.params['b1']=model_params['b1']
            self.params['W2']=model_params['W2']
            self.params['b2']=model_params['b2']
            self.params['W3']=model_params['W3']
            self.params['b3']=model_params['b3']
            self.params['W4']=model_params['W4']
            self.params['b4']=model_params['b4']
            self.params['gamma1'] = 1
            self.params['beta1'] = 0
            self.params['gamma2'] = 1
            self.params['beta2'] = 0


            # レイヤの生成
            self.layers = OrderedDict()
            self.layers['Conv1'] = Convolution(self.params['W1'], self.params['b1'],
                                               conv_param['stride'], conv_param['pad'])
            self.layers['BatchNormalization1'] = BatchNormalization(self.params['gamma1'], self.params['beta1'], rho=0.9, moving_mean=model_moving['moving_mean1'], moving_var=model_moving['moving_var1'])
            self.layers['ReLU1'] = ReLU()
            self.layers['Pool1'] = MaxPooling(pool_h=pool_size, pool_w=pool_size, stride=pool_stride, pad=pool_pad)

            self.layers['Conv2'] = Convolution(self.params['W4'], self.params['b4'],
                                               conv_param['stride'], conv_param['pad'])
            self.layers['BatchNormalization2'] = BatchNormalization(self.params['gamma2'], self.params['beta2'], rho=0.9, moving_mean=model_moving['moving_mean2'], moving_var=model_moving['moving_var2'])
            self.layers['ReLU3'] = ReLU()
            self.layers['Pool2'] = MaxPooling(pool_h=pool_size, pool_w=pool_size, stride=pool_stride, pad=pool_pad)

            #self.layers['Dropout1'] = Dropout(dropout_ratio=0.3)
            self.layers['Affine1'] = Affine(self.params['W2'], self.params['b2'])
            self.layers['ReLU2'] = ReLU()
            #self.layers['Dropout2'] = Dropout(dropout_ratio=0.3)
            self.layers['Affine2'] = Affine(self.params['W3'], self.params['b3'])

            self.last_layer = SoftmaxWithLoss()

            #self.moving = {}
            #self.moving['moving_mean1'] = self.layers['BatchNormalization1'].moving_mean
            #self.moving['moving_var1'] = self.layers['BatchNormalization1'].moving_var
            #self.moving['moving_mean2'] = self.layers['BatchNormalization2'].moving_mean
            #self.moving['moving_var2'] = self.layers['BatchNormalization2'].moving_var

        def predict(self, x):
            for layer in self.layers.values():
                x = layer.forward(x)

            return x

        def loss(self, x, t):
            """
            損失関数
            x : 入力データ
            t : 教師データ
            """
            y = self.predict(x)
            return self.last_layer.forward(y, t)

        def accuracy(self, x, t, batch_size=100):
            if t.ndim != 1 : t = np.argmax(t, axis=1)

            acc = 0.0

            for i in range(int(x.shape[0] / batch_size)):
                tx = x[i*batch_size:(i+1)*batch_size]
                tt = t[i*batch_size:(i+1)*batch_size]
                y = self.predict(tx)
                y = np.argmax(y, axis=1)
                acc += np.sum(y == tt) 

            return acc / x.shape[0]

        def gradient(self, x, t):
            """勾配を求める（誤差逆伝播法）
            Parameters
            ----------
            x : 入力データ
            t : 教師データ
            Returns
            -------
            各層の勾配を持ったディクショナリ変数
                grads['W1']、grads['W2']、...は各層の重み
                grads['b1']、grads['b2']、...は各層のバイアス
            """
            # forward
            self.loss(x, t)

            # backward
            dout = 1
            dout = self.last_layer.backward(dout)

            layers = list(self.layers.values())
            layers.reverse()
            for layer in layers:
                dout = layer.backward(dout)

            # 設定
            grads = {}
            grads['W1'], grads['b1'] = self.layers['Conv1'].dW, self.layers['Conv1'].db
            grads['W2'], grads['b2'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
            grads['W3'], grads['b3'] = self.layers['Affine2'].dW, self.layers['Affine2'].db
            grads['W4'], grads['b4'] = self.layers['Conv2'].dW, self.layers['Conv2'].db
            grads['gamma1'] = self.layers['BatchNormalization1'].dgamma
            grads['beta1'] = self.layers['BatchNormalization1'].dbeta
            grads['gamma2'] = self.layers['BatchNormalization2'].dgamma
            grads['beta2'] = self.layers['BatchNormalization2'].dbeta



            return grads

    
    snet = SimpleConvNet(input_dim1=(1, 28, 28), input_dim2=(32, 12, 12),
                     conv_param={'filter_num':32, 'filter_size':5, 'pad':0, 'stride':1},
                     pool_param={'pool_size':2, 'pad':0, 'stride':2},
                     hidden_size=500, output_size=15, weight_init_std=0.01)
    
    accuracy = snet.accuracy(test_data, test_label)
    loss  = snet.loss(test_data, test_label)
    
    return loss, accuracy # 編集不可


def main():
    """
    編集しないでください。
    """
    # テスト用データをつくる
    test_data = makedataset()

    # 予測し精度を算出する
    util.accuracy(func_predict, test_data)
    
    return


if __name__=="__main__":
    main()

0
0.0003726861170222264 1.0
1
0.007116351414542419 0.998
2
1.8818804558404629e-07 1.0
3
0.00010360550890811168 1.0
4
0.0031616409469911234 0.998
5
0.0005455819206106982 1.0
Test loss: 0.0018833423493533606
Test accuracy: 0.9993333333333333
